In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/faulty-steel-plates/faults.csv
/kaggle/input/playground-series-s4e3/sample_submission.csv
/kaggle/input/playground-series-s4e3/train.csv
/kaggle/input/playground-series-s4e3/test.csv


# XGB + Optuna

In [2]:
train = pd.read_csv('/kaggle/input/playground-series-s4e3/train.csv', encoding='utf-8')
test = pd.read_csv('/kaggle/input/playground-series-s4e3/test.csv', encoding='utf-8')
original = pd.read_csv('/kaggle/input/faulty-steel-plates/faults.csv', encoding='utf-8')
TARGET_FEATURES = ['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains','Dirtiness', 'Bumps', 'Other_Faults']
train.shape, test.shape, original.shape

((19219, 35), (12814, 28), (1941, 34))

In [3]:
train.drop(['id'],axis = 1,inplace = True)
train = pd.concat([train,original],axis = 0)
train = train.drop_duplicates()
train.reset_index(drop=True, inplace=True)
train

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,...,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,584,590,909972,909977,16,8,5,2274,113,140,...,-0.5000,-0.0104,0.1417,0,0,0,1,0,0,0
1,808,816,728350,728372,433,20,54,44478,70,111,...,0.7419,-0.2997,0.9491,0,0,0,0,0,0,1
2,39,192,2212076,2212144,11388,705,420,1311391,29,141,...,-0.0105,-0.0944,1.0000,0,0,1,0,0,0,0
3,781,789,3353146,3353173,210,16,29,3202,114,134,...,0.6667,-0.0402,0.4025,0,0,1,0,0,0,0
4,1540,1560,618457,618502,521,72,67,48231,82,111,...,0.9158,-0.2455,0.9998,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21155,249,277,325780,325796,273,54,22,35033,119,141,...,-0.4286,0.0026,0.7254,0,0,0,0,0,0,1
21156,144,175,340581,340598,287,44,24,34599,112,133,...,-0.4516,-0.0582,0.8173,0,0,0,0,0,0,1
21157,145,174,386779,386794,292,40,22,37572,120,140,...,-0.4828,0.0052,0.7079,0,0,0,0,0,0,1
21158,137,170,422497,422528,419,97,47,52715,117,140,...,-0.0606,-0.0171,0.9919,0,0,0,0,0,0,1


In [4]:
# Check targets
targets_bin = train[TARGET_FEATURES]
targets_bin[targets_bin.sum(axis=1)==2]

,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
634,0,0,1,0,0,0,1
752,0,0,1,0,0,0,1
3572,0,0,1,0,0,1,0
4416,0,0,1,0,0,0,1
4530,0,0,1,0,0,0,1
4664,0,0,1,0,0,0,1
6448,0,0,1,0,0,0,1
7589,0,0,1,0,0,0,1
8075,0,0,1,0,0,0,1
8538,0,0,1,0,0,0,1


In [5]:
train['Target'] = np.argmax(train[TARGET_FEATURES].values, axis=1) + 1
train.loc[train[TARGET_FEATURES].sum(axis=1) == 0, 'Target'] = 0
train.drop(TARGET_FEATURES, inplace=True,axis =1)
train

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,...,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Target
0,584,590,909972,909977,16,8,5,2274,113,140,...,1.0000,1.0000,0.0,1.2041,0.9031,0.6990,-0.5000,-0.0104,0.1417,4
1,808,816,728350,728372,433,20,54,44478,70,111,...,0.2500,1.0000,1.0,2.6365,0.7782,1.7324,0.7419,-0.2997,0.9491,7
2,39,192,2212076,2212144,11388,705,420,1311391,29,141,...,0.2363,0.3857,0.0,4.0564,2.1790,2.2095,-0.0105,-0.0944,1.0000,3
3,781,789,3353146,3353173,210,16,29,3202,114,134,...,0.3750,0.9310,1.0,2.3222,0.7782,1.4314,0.6667,-0.0402,0.4025,3
4,1540,1560,618457,618502,521,72,67,48231,82,111,...,0.2105,0.9861,1.0,2.7694,1.4150,1.8808,0.9158,-0.2455,0.9998,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21155,249,277,325780,325796,273,54,22,35033,119,141,...,0.5185,0.7273,0.0,2.4362,1.4472,1.2041,-0.4286,0.0026,0.7254,7
21156,144,175,340581,340598,287,44,24,34599,112,133,...,0.7046,0.7083,0.0,2.4579,1.4914,1.2305,-0.4516,-0.0582,0.8173,7
21157,145,174,386779,386794,292,40,22,37572,120,140,...,0.7250,0.6818,0.0,2.4654,1.4624,1.1761,-0.4828,0.0052,0.7079,7
21158,137,170,422497,422528,419,97,47,52715,117,140,...,0.3402,0.6596,0.0,2.6222,1.5185,1.4914,-0.0606,-0.0171,0.9919,7


In [6]:
import numpy as np
import optuna
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, StratifiedKFold
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [7]:
X = train.drop(['Target'], axis=1)  
y = train['Target']  

In [8]:
RETRAIN_MODEL = True
def objective(trial):
    param = {
        'objective':'multi:softmax',
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.05),
        'n_estimators': trial.suggest_int('n_estimators',250,1000),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0,log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0,log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 1.0),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'device' : "cuda",
        'tree_method':"hist"
    }

    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    auc_scores = []

    for train_idx, valid_idx in cv.split(X, y):

        X_train_fold, X_valid_fold = X.iloc[train_idx], X.iloc[valid_idx]
        y_train_fold, y_valid_fold = y.iloc[train_idx], y.iloc[valid_idx]

        model = XGBClassifier(**param)
        model.fit(X_train_fold, y_train_fold)

        y_prob = model.predict_proba(X_valid_fold)

        average_auc = roc_auc_score(targets_bin.iloc[valid_idx], y_prob[:, 1:], multi_class="ovr", average="macro")
        auc_scores.append(average_auc)

    return np.mean(auc_scores)

study = optuna.create_study(direction='maximize',study_name = "xgb_model_training")

if RETRAIN_MODEL:
    study.optimize(objective, n_trials=100)

    print(f"Best trial average AUC: {study.best_value:.4f}")
    for key, value in study.best_params.items():
        print(f"{key}: {value}")

[I 2024-03-09 14:40:53,940] A new study created in memory with name: xgb_model_training
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [14:41:16] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
[I 2024-03-09 14:44:42,456] Trial 0 finished with value: 0.8977401379416714 and parameters: {'learning_rate': 0.005219114924743408, 'n_estimators': 845, 'reg_alpha': 4.1208708173103436e-07, 'reg_lambda': 7.383739881532837e-07, 'max_depth': 8, 'colsample_bytree': 0.7721306125437644, 'subsample': 0.8034210605432881, 'min_child_weight': 3}. Best is trial 0 

Best trial average AUC: 0.8994
learning_rate: 0.01653460618062825
n_estimators: 701
reg_alpha: 7.273708238403666e-08
reg_lambda: 0.005460603687341128
max_depth: 5
colsample_bytree: 0.35776753567288
subsample: 0.9532191723985007
min_child_weight: 5


In [13]:
# Best param
best_params = {
    'objective':'multi:softmax',
    'learning_rate': 0.01653460618062825,
    'n_estimators': 701,
    'reg_alpha': 7.273708238403666e-08,
    'reg_lambda': 0.005460603687341128,
    'max_depth': 5,
    'colsample_bytree': 0.35776753567288,
    'subsample': 0.9532191723985007,
    'min_child_weight': 5,
    'device' : "cuda",
'tree_method':"hist"
}

In [9]:
test.drop(['id'],inplace = True,axis = 1)

In [15]:
cv = StratifiedKFold(n_splits=20, shuffle=True, random_state=42)
auc_scores = []
y_prob_test = []
for train_idx, valid_idx in cv.split(X, y):
    X_train_fold, X_valid_fold = X.iloc[train_idx], X.iloc[valid_idx]
    y_train_fold, y_valid_fold = y.iloc[train_idx], y.iloc[valid_idx]

    model = XGBClassifier(**best_params)
    model.fit(X_train_fold, y_train_fold)

    y_prob = model.predict_proba(X_valid_fold)
    
    y_prob_test.append(model.predict_proba(test))

    average_auc = roc_auc_score(targets_bin.iloc[valid_idx], y_prob[:, 1:], multi_class="ovr", average="macro")
    auc_scores.append(average_auc)

print(np.mean(auc_scores))
y_prob_test_array = np.array(y_prob_test)
pred = np.mean(y_prob_test_array, axis=0)

0.8995422993078691


In [16]:
sample_prediction = pd.read_csv('/kaggle/input/playground-series-s4e3/sample_submission.csv')

In [17]:
predictions = pd.DataFrame(pred[:, 1:], columns=[TARGET_FEATURES])
predictions['id'] = sample_prediction['id']
predictions.to_csv('20240310_XGB_Tuned_submission.csv', index=False)